<a href="https://colab.research.google.com/github/alouvre/stunting/blob/main/Stunting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

data kaggle = https://www.kaggle.com/datasets/uniksh17/dataset-posyandu-mawar

In [1]:
# !git clone https://github.com/alouvre/stunting.git

Cloning into 'stunting'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (15/15), 421.22 KiB | 2.45 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
!ls stunting

data_posyandu  data_posyandu.zip  data_unicef  README.md


In [44]:
import pandas as pd
import numpy as np
from scipy import stats

In [82]:
data = pd.read_excel('stunting/data_posyandu/data posyandu bulan november 2020.xlsx')
data.head()

,Unnamed: 0,data posyandu rt.01/rw.03 candimulyo,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,subject,umur,berat badan (kg),tinggi badan (cm)
1,NaN,1,3 tahun,12,80
2,NaN,2,4 tahun,18,79
3,NaN,3,1 tahun,11,76
4,NaN,4,6 bulan,7,69


In [83]:
data = data.drop('Unnamed: 0', axis=1, errors='ignore')

In [84]:
data.head()

,data posyandu rt.01/rw.03 candimulyo,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,subject,umur,berat badan (kg),tinggi badan (cm)
1,1,3 tahun,12,80
2,2,4 tahun,18,79
3,3,1 tahun,11,76
4,4,6 bulan,7,69


In [85]:
new_column_names = ['subject', 'umur', 'berat badan (kg)', 'tinggi badan (cm)']
data.columns = new_column_names

In [86]:
data.head()

,subject,umur,berat badan (kg),tinggi badan (cm)
0,subject,umur,berat badan (kg),tinggi badan (cm)
1,1,3 tahun,12,80
2,2,4 tahun,18,79
3,3,1 tahun,11,76
4,4,6 bulan,7,69


In [87]:
data = data.drop(0)

In [88]:
data.head()

,subject,umur,berat badan (kg),tinggi badan (cm)
1,1,3 tahun,12,80
2,2,4 tahun,18,79
3,3,1 tahun,11,76
4,4,6 bulan,7,69
5,5,5 tahun,15,102


In [90]:
# Fungsi untuk mengonversi umur ke dalam bulan
def convert_umur_to_bulan(umur):
    umur_split = umur.split()
    if len(umur_split) == 2:
        if 'tahun' in umur_split[1].lower():
            return int(umur_split[0]) * 12
        elif 'bulan' in umur_split[1].lower():
            return int(umur_split[0])
    elif len(umur_split) == 1 and umur_split[0].lower() == 'bulan':
        return 1  # Misalkan 1 bulan untuk umur yang hanya disebutkan dalam bulan
    return None

# Mengonversi kolom 'umur' ke dalam bulan
data['umur_bulan'] = data['umur'].apply(convert_umur_to_bulan)

In [91]:
# Ubah tipe data kolom 'umur', 'berat badan (kg)', dan 'tinggi badan (cm)' menjadi numerik
data['umur'] = pd.to_numeric(data['umur'], errors='coerce')
data['berat badan (kg)'] = pd.to_numeric(data['berat badan (kg)'], errors='coerce')
data['tinggi badan (cm)'] = pd.to_numeric(data['tinggi badan (cm)'], errors='coerce')

In [92]:
data.head()

,subject,umur,berat badan (kg),tinggi badan (cm),umur_bulan
1,1,NaN,12,80,36
2,2,NaN,18,79,48
3,3,NaN,11,76,12
4,4,NaN,7,69,6
5,5,NaN,15,102,60


In [93]:
# Fungsi untuk menghitung Z-Score TB/U berdasarkan umur, berat badan, dan tinggi badan
def calculate_z_score_umur_berat_badan_tinggi_badan(umur, berat_badan, tinggi_badan):
    # Menghitung Z-Score untuk umur, berat badan, dan tinggi badan
    z_score_umur = stats.zscore(umur)
    z_score_berat_badan = stats.zscore(berat_badan)
    z_score_tinggi_badan = stats.zscore(tinggi_badan)

    # Tambahkan rumus yang sesuai untuk menghitung Z-Score TB/U berdasarkan umur, berat badan, dan tinggi badan
    z_score_tb_u = z_score_umur + z_score_berat_badan + z_score_tinggi_badan
    return z_score_tb_u

# Menentukan batas -2 SD untuk mendefinisikan stunting
batas_stunting = -2

# Menambahkan kolom 'Z-Score TB/U' dan 'Label' ke DataFrame
data['Z-Score TB/U'] = calculate_z_score_umur_berat_badan_tinggi_badan(data['umur'], data['berat badan (kg)'], data['tinggi badan (cm)'])
data['Label'] = data['Z-Score TB/U'].apply(lambda x: 'Stunting' if x < batas_stunting else 'Bukan Stunting')

# Menampilkan DataFrame dengan kolom label
data

,subject,umur,berat badan (kg),tinggi badan (cm),umur_bulan,Z-Score TB/U,Label
1,1,NaN,12,80,36,NaN,Bukan Stunting
2,2,NaN,18,79,48,NaN,Bukan Stunting
3,3,NaN,11,76,12,NaN,Bukan Stunting
4,4,NaN,7,69,6,NaN,Bukan Stunting
5,5,NaN,15,102,60,NaN,Bukan Stunting
6,6,NaN,11,99,36,NaN,Bukan Stunting
7,7,NaN,10,97,24,NaN,Bukan Stunting
8,8,NaN,13,104,48,NaN,Bukan Stunting
9,9,NaN,9,96,24,NaN,Bukan Stunting
10,10,NaN,10,89,24,NaN,Bukan Stunting
